In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go

In [2]:
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")

tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [3]:
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
headers = {"User-Agent": "Mozilla/5.0"}
html_data = requests.get(url, headers=headers).text
tables = pd.read_html(html_data)
tesla_revenue = tables[1]
tesla_revenue.columns = ["Date", "Revenue"]
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace(r'[\$,]', '', regex=True)
tesla_revenue = tesla_revenue[tesla_revenue["Revenue"] != ""]
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].astype(float)

tesla_revenue.head()

C:\Users\Modit\AppData\Local\Temp\ipykernel_18928\4011701627.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html_data)


,Date,Revenue
0,2025-06-30,22496.0
1,2025-03-31,19335.0
2,2024-12-31,25707.0
3,2024-09-30,25182.0
4,2024-06-30,25500.0


In [4]:
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693349,1.603295,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0


In [5]:
url_gme = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
headers = {"User-Agent": "Mozilla/5.0"}
html_data_gme = requests.get(url_gme, headers=headers).text
tables_gme = pd.read_html(html_data_gme)
gme_revenue = tables_gme[1]
gme_revenue.columns = ["Date", "Revenue"]
gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace(r'[\$,]', '', regex=True)
gme_revenue = gme_revenue[gme_revenue["Revenue"] != ""]
gme_revenue["Revenue"] = gme_revenue["Revenue"].astype(float)

gme_revenue.head()

C:\Users\Modit\AppData\Local\Temp\ipykernel_18928\1110480697.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables_gme = pd.read_html(html_data_gme)


,Date,Revenue
0,2025-07-31,972.0
1,2025-04-30,732.0
2,2025-01-31,1283.0
3,2024-10-31,860.0
4,2024-07-31,798.0


In [6]:
def make_graph(stock_data, revenue_data, title):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data['Close'],
                             name="Stock Price", mode='lines'))

    fig.add_trace(go.Scatter(x=revenue_data['Date'], y=revenue_data['Revenue'],
                             name="Revenue (Millions)", mode='lines', yaxis="y2"))

    fig.update_layout(
        title=title,
        xaxis_title="Date",
        yaxis=dict(title="Stock Price (USD)", side="left"),
        yaxis2=dict(title="Revenue (Millions USD)", overlaying="y", side="right"),
        legend=dict(x=0.1, y=0.9)
    )
    fig.show()

make_graph(tesla_data, tesla_revenue, "Tesla Stock Price vs Revenue")


In [7]:
make_graph(gme_data, gme_revenue, "GameStop Stock Price vs Revenue")